# Axona to NWB

Convert Axona raw (`.bin` + `.set`) or unit data (`.X`, `.pos`, `.eeg`, `.egf`) to NWB format. 

It might be handy to convert the position data in the `.bin` file to a `.pos` file, allowing us to use the same code for both axona formats, as well as the Intan format later on (requiring the Hussaini lab to create their own `.pos` files based on the position tracking used). 

### Using nwb-conversion-tools directly

It seems that I cannot use the datainterfaces directly (at least not AxonaRecordingExtractorInterface), because I get a datetime error: 

`TypeError: NWBFile.__init__: incorrect type for 'session_start_time' (got 'str', expected 'datetime')`. 

This is somewhat annoying, since when using NWBConverters it should be an ISO datetime string. Instead, let's simply create Converters for each situation and not use datainterfaces directly.

In [1]:
import json
from pathlib import Path

import matplotlib.pyplot as plt
from pynwb import NWBHDF5IO
from nwbwidgets import nwb2widget

In [2]:
from nwb_conversion_tools import (
    NWBConverter,
    AxonaRecordingExtractorInterface,
    AxonaUnitRecordingExtractorInterface,
    AxonaPositionDataInterface,
    AxonaLFPDataInterface
)

In [3]:
# Make sure to select a set-filename for which we also have raw, tetrode, eeg, egf and pos data.

filename = '/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/sample_bin_to_tint/axona_sample.set'
filename_eeg = filename.replace('.set', '.eeg')
filename_egf = filename.replace('.set', '.egf')

base_dir = Path(filename).parent

In [4]:
# Make NWBConverters from datainterfaces for sensible combinations of datainterfaces

class HussainiBinNWBConverter(NWBConverter):
    data_interface_classes = dict(
        AxonaRecordingExtractorInterface=AxonaRecordingExtractorInterface
    )


class HussainiBinPosLfpNWBConverter(NWBConverter):
    data_interface_classes = dict(
        AxonaRecordingExtractorInterface=AxonaRecordingExtractorInterface,
        AxonaPositionDataInterface=AxonaPositionDataInterface,
        AxonaLFPDataInterface=AxonaLFPDataInterface
    )

    
class HussainiTetrodeNWBConverter(NWBConverter):
    data_interface_classes = dict(
        AxonaUnitRecordingExtractorInterface=AxonaUnitRecordingExtractorInterface
    )
    
    
class HussainiPosNWBConverter(NWBConverter):
    data_interface_classes = dict(
        AxonaPositionDataInterface=AxonaPositionDataInterface
    )


class HussainiLfpNWBConverter(NWBConverter):
    data_interface_classes = dict(
        AxonaLFPDataInterface=AxonaLFPDataInterface
    )


class HussainiUnitNWBConverter(NWBConverter):
    data_interface_classes = dict(
        AxonaUnitRecordingExtractorInterface=AxonaUnitRecordingExtractorInterface,
        AxonaPositionDataInterface=AxonaPositionDataInterface,
        AxonaLFPDataInterface=AxonaLFPDataInterface
    )

1. __HussainiBinPosLfpNWBConverter__: `.bin` + `.pos` + `.eeg` + `.set`

In [5]:
#1. __HussainiBinPosLfpNWBConverter__: `.bin` + `.pos` + `.eeg` + `.set`

# Set nwbfile name
nwbfile_name = 'nwb_test_bin_pos_lfp.nwb'


# Specify source data
source_data = dict(
    AxonaLFPDataInterface=dict(filename=filename_eeg),
    AxonaRecordingExtractorInterface=dict(filename=filename),
    AxonaPositionDataInterface=dict(filename=filename),
)
print(json.dumps(source_data, indent=2))


# Initialize Converter
converter = HussainiBinPosLfpNWBConverter(source_data=source_data)


# Get metadata
metadata = converter.get_metadata()


# Export to NWB file
output_file = base_dir / nwbfile_name

converter.run_conversion(
    metadata=metadata,
    nwbfile_path=output_file,
    overwrite=True,
    save_to_file=True,
    conversion_options=None
)

{
  "AxonaLFPDataInterface": {
    "filename": "/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/sample_bin_to_tint/axona_sample.eeg"
  },
  "AxonaRecordingExtractorInterface": {
    "filename": "/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/sample_bin_to_tint/axona_sample.set"
  },
  "AxonaPositionDataInterface": {
    "filename": "/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/sample_bin_to_tint/axona_sample.set"
  }
}
Source data is valid!
Metadata is valid!


/home/sbuergers/spikeinterface/hussaini-test-pipeline/venv/lib/python3.8/site-packages/pynwb/file.py:801: UserWarning: Date is missing timezone information. Updating to local timezone.
  warn("Date is missing timezone information. Updating to local timezone.")
/home/sbuergers/spikeinterface/hussaini-test-pipeline/venv/lib/python3.8/site-packages/hdmf/common/table.py:447: UserWarning: An attribute 'name' already exists on DynamicTable 'electrodes' so this column cannot be accessed as an attribute, e.g., table.name; it can only be accessed using other methods, e.g., table['name'].
  warn(msg)
/home/sbuergers/spikeinterface/nwb-conversion-tools/nwb_conversion_tools/utils/spike_interface.py:302: UserWarning: cannot create electrodes for this recording as ids already exist
  warnings.warn("cannot create electrodes for this recording as ids already exist")
/home/sbuergers/spikeinterface/hussaini-test-pipeline/venv/lib/python3.8/site-packages/spikeextractors/extraction_tools.py:802: UserWarni

NWB file saved at /mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/sample_bin_to_tint/nwb_test_bin_pos_lfp.nwb!


In [6]:
# Check NWB file

fname = output_file
with NWBHDF5IO(fname, 'r') as io:
    nwbfile = io.read()
    print(nwbfile)
    

%matplotlib inline

io = NWBHDF5IO(output_file, mode='r')
nwb = io.read()

nwb2widget(nwb)

root pynwb.file.NWBFile at 0x140662105252816
Fields:
  acquisition: {
    ElectricalSeries_raw <class 'pynwb.ecephys.ElectricalSeries'>
  }
  devices: {
    Axona <class 'pynwb.device.Device'>
  }
  electrode_groups: {
    Group0 <class 'pynwb.ecephys.ElectrodeGroup'>,
    Group1 <class 'pynwb.ecephys.ElectrodeGroup'>,
    Group2 <class 'pynwb.ecephys.ElectrodeGroup'>,
    Group3 <class 'pynwb.ecephys.ElectrodeGroup'>
  }
  electrodes: electrodes <class 'hdmf.common.table.DynamicTable'>
  experimenter: ['Abid']
  file_create_date: [datetime.datetime(2021, 8, 7, 12, 56, 8, 138206, tzinfo=tzoffset(None, 7200))]
  identifier: a2e4afe6-c434-47c7-8c89-90548c8a1304
  processing: {
    behavior <class 'pynwb.base.ProcessingModule'>,
    ecephys <class 'pynwb.base.ProcessingModule'>
  }
  session_start_time: 2020-10-04 11:07:07+02:00
  timestamps_reference_time: 2020-10-04 11:07:07+02:00



2. __HussainiBinNWBConverter__: `.bin` + `.set`

In [7]:
#1. __HussainiBinPosLfpNWBConverter__: `.bin` + `.set`

# Set nwbfile name
nwbfile_name = 'nwb_test_bin.nwb'


# Specify source data
source_data = dict(
    AxonaRecordingExtractorInterface=dict(filename=filename)
)
print(json.dumps(source_data, indent=2))


# Initialize Converter
converter = HussainiBinNWBConverter(source_data=source_data)


# Get metadata
metadata = converter.get_metadata()


# Export to NWB file
output_file = base_dir / nwbfile_name

converter.run_conversion(
    metadata=metadata,
    nwbfile_path=output_file,
    overwrite=True,
    save_to_file=True,
    conversion_options=None
)


# Check NWB file

fname = output_file
with NWBHDF5IO(fname, 'r') as io:
    nwbfile = io.read()
    print(nwbfile)
    

%matplotlib inline

io = NWBHDF5IO(output_file, mode='r')
nwb = io.read()

nwb2widget(nwb)

{
  "AxonaRecordingExtractorInterface": {
    "filename": "/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/sample_bin_to_tint/axona_sample.set"
  }
}
Source data is valid!
Metadata is valid!
NWB file saved at /mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/sample_bin_to_tint/nwb_test_bin.nwb!
root pynwb.file.NWBFile at 0x140661462403392
Fields:
  acquisition: {
    ElectricalSeries_raw <class 'pynwb.ecephys.ElectricalSeries'>
  }
  devices: {
    Axona <class 'pynwb.device.Device'>
  }
  electrode_groups: {
    Group0 <class 'pynwb.ecephys.ElectrodeGroup'>,
    Group1 <class 'pynwb.ecephys.ElectrodeGroup'>,
    Group2 <class 'pynwb.ecephys.ElectrodeGroup'>,
    Group3 <class 'pynwb.ecephys.ElectrodeGroup'>
  }
  electrodes: electrodes <class 'hdmf.common.table.DynamicTable'>
  experimenter: ['Abid']
  file_create_date: [datetime.datetime(2021, 8, 7, 12, 56, 11, 66643, tzinfo=tzoffset(None, 7200))]
  identifier: 32303dbe-e9c6-4232-81bb-9cfb4bc7fab3
  session_start_time

3. __HussainiTetrodeNWBConverter__: `.X` + `.set`

In [8]:
# Set nwbfile name
nwbfile_name = 'nwb_test_tetrode.nwb'


# Specify source data
source_data = dict(
    AxonaUnitRecordingExtractorInterface=dict(filename=filename)
)
print(json.dumps(source_data, indent=2))


# Initialize Converter
converter = HussainiTetrodeNWBConverter(source_data=source_data)


# Get metadata
metadata = converter.get_metadata()


# Export to NWB file
output_file = base_dir / nwbfile_name

converter.run_conversion(
    metadata=metadata,
    nwbfile_path=output_file,
    overwrite=True,
    save_to_file=True,
    conversion_options=None
)


# Check NWB file

fname = output_file
with NWBHDF5IO(fname, 'r') as io:
    nwbfile = io.read()
    print(nwbfile)
    

%matplotlib inline

io = NWBHDF5IO(output_file, mode='r')
nwb = io.read()

nwb2widget(nwb)

{
  "AxonaUnitRecordingExtractorInterface": {
    "filename": "/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/sample_bin_to_tint/axona_sample.set"
  }
}
Source data is valid!
Metadata is valid!
NWB file saved at /mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/sample_bin_to_tint/nwb_test_tetrode.nwb!
root pynwb.file.NWBFile at 0x140661454900768
Fields:
  acquisition: {
    ElectricalSeries_raw <class 'pynwb.ecephys.ElectricalSeries'>
  }
  devices: {
    Axona <class 'pynwb.device.Device'>
  }
  electrode_groups: {
    Group0 <class 'pynwb.ecephys.ElectrodeGroup'>,
    Group1 <class 'pynwb.ecephys.ElectrodeGroup'>,
    Group2 <class 'pynwb.ecephys.ElectrodeGroup'>,
    Group3 <class 'pynwb.ecephys.ElectrodeGroup'>
  }
  electrodes: electrodes <class 'hdmf.common.table.DynamicTable'>
  experimenter: ['Abid']
  file_create_date: [datetime.datetime(2021, 8, 7, 12, 56, 13, 346585, tzinfo=tzoffset(None, 7200))]
  identifier: 20648916-d51a-484d-a9bb-ec7bd67341dd
  session_s

4. __HussainiPosNWBConverter__: `.pos` + `.set`

In [9]:

# Set nwbfile name
nwbfile_name = 'nwb_test_pos.nwb'


# Specify source data
source_data = dict(
    AxonaPositionDataInterface=dict(filename=filename),
)
print(json.dumps(source_data, indent=2))


# Initialize Converter
converter = HussainiPosNWBConverter(source_data=source_data)


# Get metadata
metadata = converter.get_metadata()


# Export to NWB file
output_file = base_dir / nwbfile_name

converter.run_conversion(
    metadata=metadata,
    nwbfile_path=output_file,
    overwrite=True,
    save_to_file=True,
    conversion_options=None
)


# Check NWB file

fname = output_file
with NWBHDF5IO(fname, 'r') as io:
    nwbfile = io.read()
    print(nwbfile)
    

%matplotlib inline

io = NWBHDF5IO(output_file, mode='r')
nwb = io.read()

nwb2widget(nwb)

{
  "AxonaPositionDataInterface": {
    "filename": "/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/sample_bin_to_tint/axona_sample.set"
  }
}
Source data is valid!
Metadata is valid!
NWB file saved at /mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/sample_bin_to_tint/nwb_test_pos.nwb!
root pynwb.file.NWBFile at 0x140661462259888
Fields:
  file_create_date: [datetime.datetime(2021, 8, 7, 12, 56, 15, 302932, tzinfo=tzoffset(None, 7200))]
  identifier: 39e076c5-6d26-4f90-8c6d-9502ae74c2f9
  processing: {
    behavior <class 'pynwb.base.ProcessingModule'>
  }
  session_description: no description
  session_start_time: 1970-01-01 00:00:00+01:00
  timestamps_reference_time: 1970-01-01 00:00:00+01:00



5. __HussainiLfpNWBConverter__: `.eeg` + `.set`


In [10]:

# Set nwbfile name
nwbfile_name = 'nwb_test_lfp_eeg.nwb'


# Specify source data
source_data = dict(
    AxonaLFPDataInterface=dict(filename=filename_eeg)
)
print(json.dumps(source_data, indent=2))


# Initialize Converter
converter = HussainiLfpNWBConverter(source_data=source_data)


# Get metadata
metadata = converter.get_metadata()


# Export to NWB file
output_file = base_dir / nwbfile_name

converter.run_conversion(
    metadata=metadata,
    nwbfile_path=output_file,
    overwrite=True,
    save_to_file=True,
    conversion_options=None
)


# Check NWB file

fname = output_file
with NWBHDF5IO(fname, 'r') as io:
    nwbfile = io.read()
    print(nwbfile)
    

%matplotlib inline

io = NWBHDF5IO(output_file, mode='r')
nwb = io.read()

nwb2widget(nwb)

{
  "AxonaLFPDataInterface": {
    "filename": "/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/sample_bin_to_tint/axona_sample.eeg"
  }
}
Source data is valid!
Metadata is valid!
NWB file saved at /mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/sample_bin_to_tint/nwb_test_lfp_eeg.nwb!
root pynwb.file.NWBFile at 0x140661471265936
Fields:
  devices: {
    Axona <class 'pynwb.device.Device'>
  }
  electrode_groups: {
    Group0 <class 'pynwb.ecephys.ElectrodeGroup'>
  }
  electrodes: electrodes <class 'hdmf.common.table.DynamicTable'>
  experimenter: ['Abid']
  file_create_date: [datetime.datetime(2021, 8, 7, 12, 56, 17, 590463, tzinfo=tzoffset(None, 7200))]
  identifier: e730cfa0-b77f-4a46-9e5e-ea5ddb518ef6
  processing: {
    ecephys <class 'pynwb.base.ProcessingModule'>
  }
  session_start_time: 2020-10-04 11:07:07+02:00
  timestamps_reference_time: 2020-10-04 11:07:07+02:00



6. __HussainiLfpNWBConverter__: `.egf` + `.set`


In [11]:

# Set nwbfile name
nwbfile_name = 'nwb_test_lfp_egf.nwb'


# Specify source data
source_data = dict(
    AxonaLFPDataInterface=dict(filename=filename_egf)
)
print(json.dumps(source_data, indent=2))


# Initialize Converter
converter = HussainiLfpNWBConverter(source_data=source_data)


# Get metadata
metadata = converter.get_metadata()


# Export to NWB file
output_file = base_dir / nwbfile_name

converter.run_conversion(
    metadata=metadata,
    nwbfile_path=output_file,
    overwrite=True,
    save_to_file=True,
    conversion_options=None
)


# Check NWB file

fname = output_file
with NWBHDF5IO(fname, 'r') as io:
    nwbfile = io.read()
    print(nwbfile)
    

%matplotlib inline

io = NWBHDF5IO(output_file, mode='r')
nwb = io.read()

nwb2widget(nwb)

{
  "AxonaLFPDataInterface": {
    "filename": "/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/sample_bin_to_tint/axona_sample.egf"
  }
}
Source data is valid!
Metadata is valid!
NWB file saved at /mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/sample_bin_to_tint/nwb_test_lfp_egf.nwb!
root pynwb.file.NWBFile at 0x140661469818544
Fields:
  devices: {
    Axona <class 'pynwb.device.Device'>
  }
  electrode_groups: {
    Group0 <class 'pynwb.ecephys.ElectrodeGroup'>
  }
  electrodes: electrodes <class 'hdmf.common.table.DynamicTable'>
  experimenter: ['Abid']
  file_create_date: [datetime.datetime(2021, 8, 7, 12, 56, 20, 731439, tzinfo=tzoffset(None, 7200))]
  identifier: 9d3b5dc4-c59b-40cd-a065-c073db88370c
  processing: {
    ecephys <class 'pynwb.base.ProcessingModule'>
  }
  session_start_time: 2020-10-04 11:07:07+02:00
  timestamps_reference_time: 2020-10-04 11:07:07+02:00



7. __HussainiUnitNWBConverter__: `.X` + `.pos` + `.eeg` + `.set`

In [12]:

# Set nwbfile name
nwbfile_name = 'nwb_test_unit_eeg.nwb'


# Specify source data
source_data = dict(
    AxonaLFPDataInterface=dict(filename=filename_eeg),
    AxonaUnitRecordingExtractorInterface=dict(filename=filename),
    AxonaPositionDataInterface=dict(filename=filename),
)
print(json.dumps(source_data, indent=2))


# Initialize Converter
converter = HussainiUnitNWBConverter(source_data=source_data)


# Get metadata
metadata = converter.get_metadata()


# Export to NWB file
output_file = base_dir / nwbfile_name

converter.run_conversion(
    metadata=metadata,
    nwbfile_path=output_file,
    overwrite=True,
    save_to_file=True,
    conversion_options=None
)


# Check NWB file

fname = output_file
with NWBHDF5IO(fname, 'r') as io:
    nwbfile = io.read()
    print(nwbfile)
    

%matplotlib inline

io = NWBHDF5IO(output_file, mode='r')
nwb = io.read()

nwb2widget(nwb)

{
  "AxonaLFPDataInterface": {
    "filename": "/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/sample_bin_to_tint/axona_sample.eeg"
  },
  "AxonaUnitRecordingExtractorInterface": {
    "filename": "/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/sample_bin_to_tint/axona_sample.set"
  },
  "AxonaPositionDataInterface": {
    "filename": "/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/sample_bin_to_tint/axona_sample.set"
  }
}
Source data is valid!
Metadata is valid!
NWB file saved at /mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/sample_bin_to_tint/nwb_test_unit_eeg.nwb!
root pynwb.file.NWBFile at 0x140661466742544
Fields:
  acquisition: {
    ElectricalSeries_raw <class 'pynwb.ecephys.ElectricalSeries'>
  }
  devices: {
    Axona <class 'pynwb.device.Device'>
  }
  electrode_groups: {
    Group0 <class 'pynwb.ecephys.ElectrodeGroup'>,
    Group1 <class 'pynwb.ecephys.ElectrodeGroup'>,
    Group2 <class 'pynwb.ecephys.ElectrodeGroup'>,
    Group3 <

8. __HussainiUnitNWBConverter__: `.X` + `.pos` + `.egf` + `.set`

In [13]:

# Set nwbfile name
nwbfile_name = 'nwb_test_unit_egf.nwb'


# Specify source data
source_data = dict(
    AxonaLFPDataInterface=dict(filename=filename_egf),
    AxonaUnitRecordingExtractorInterface=dict(filename=filename),
    AxonaPositionDataInterface=dict(filename=filename),
)
print(json.dumps(source_data, indent=2))


# Initialize Converter
converter = HussainiUnitNWBConverter(source_data=source_data)


# Get metadata
metadata = converter.get_metadata()


# Export to NWB file
output_file = base_dir / nwbfile_name

converter.run_conversion(
    metadata=metadata,
    nwbfile_path=output_file,
    overwrite=True,
    save_to_file=True,
    conversion_options=None
)


# Check NWB file

fname = output_file
with NWBHDF5IO(fname, 'r') as io:
    nwbfile = io.read()
    print(nwbfile)
    

%matplotlib inline

io = NWBHDF5IO(output_file, mode='r')
nwb = io.read()

nwb2widget(nwb)

{
  "AxonaLFPDataInterface": {
    "filename": "/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/sample_bin_to_tint/axona_sample.egf"
  },
  "AxonaUnitRecordingExtractorInterface": {
    "filename": "/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/sample_bin_to_tint/axona_sample.set"
  },
  "AxonaPositionDataInterface": {
    "filename": "/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/sample_bin_to_tint/axona_sample.set"
  }
}
Source data is valid!
Metadata is valid!
NWB file saved at /mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/sample_bin_to_tint/nwb_test_unit_egf.nwb!
root pynwb.file.NWBFile at 0x140661457855488
Fields:
  acquisition: {
    ElectricalSeries_raw <class 'pynwb.ecephys.ElectricalSeries'>
  }
  devices: {
    Axona <class 'pynwb.device.Device'>
  }
  electrode_groups: {
    Group0 <class 'pynwb.ecephys.ElectrodeGroup'>,
    Group1 <class 'pynwb.ecephys.ElectrodeGroup'>,
    Group2 <class 'pynwb.ecephys.ElectrodeGroup'>,
    Group3 <

## Adding data to existing nwb files

1. Add `.pos` and `.eeg` data to `.bin` acquisition

In [5]:

# Set nwbfile name
nwbfile_name = 'nwb_test_bin.nwb'


# Specify source data for pos
source_data = dict(
    AxonaPositionDataInterface=dict(filename=filename),
)
print(json.dumps(source_data, indent=2))


# Initialize Converter
converter = HussainiPosNWBConverter(source_data=source_data)


# Get metadata
metadata = converter.get_metadata()


# Export to NWB file
output_file = base_dir / nwbfile_name

converter.run_conversion(
    metadata=metadata,
    nwbfile_path=output_file,
    overwrite=False,
    save_to_file=True,
    conversion_options=None
)



# Specify source data for eeg
source_data = dict(
    AxonaLFPDataInterface=dict(filename=filename_eeg),
)
print(json.dumps(source_data, indent=2))


# Initialize Converter
converter = HussainiLfpNWBConverter(source_data=source_data)


# Get metadata
metadata = converter.get_metadata()


# Export to NWB file
output_file = base_dir / nwbfile_name

converter.run_conversion(
    metadata=metadata,
    nwbfile_path=output_file,
    overwrite=False,
    save_to_file=True,
    conversion_options=None
)


# Check NWB file

fname = output_file
with NWBHDF5IO(fname, 'r') as io:
    nwbfile = io.read()
    print(nwbfile)
    

%matplotlib inline

io = NWBHDF5IO(output_file, mode='r')
nwb = io.read()

nwb2widget(nwb)

{
  "AxonaPositionDataInterface": {
    "filename": "/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/sample_bin_to_tint/axona_sample.set"
  }
}
Source data is valid!
Metadata is valid!


/home/sbuergers/spikeinterface/hussaini-test-pipeline/venv/lib/python3.8/site-packages/hdmf/common/table.py:447: UserWarning: An attribute 'name' already exists on DynamicTable 'electrodes' so this column cannot be accessed as an attribute, e.g., table.name; it can only be accessed using other methods, e.g., table['name'].
  warn(msg)
/home/sbuergers/spikeinterface/hussaini-test-pipeline/venv/lib/python3.8/site-packages/hdmf/build/objectmapper.py:256: DtypeConversionWarning: Spec 'SpatialSeries/timestamps': Value with data type int64 is being converted to data type float64 as specified.
  warnings.warn(full_warning_msg, DtypeConversionWarning)


NWB file saved at /mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/sample_bin_to_tint/nwb_test_bin.nwb!
{
  "AxonaLFPDataInterface": {
    "filename": "/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/sample_bin_to_tint/axona_sample.eeg"
  }
}
Source data is valid!
Metadata is valid!


/home/sbuergers/spikeinterface/nwb-conversion-tools/nwb_conversion_tools/utils/spike_interface.py:302: UserWarning: cannot create electrodes for this recording as ids already exist
  warnings.warn("cannot create electrodes for this recording as ids already exist")
/home/sbuergers/spikeinterface/hussaini-test-pipeline/venv/lib/python3.8/site-packages/spikeextractors/extraction_tools.py:802: UserWarning: The recording extractor does not have unscaled traces. Returning scaled traces
  warnings.warn("The recording extractor does not have unscaled traces. Returning scaled traces")


NWB file saved at /mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/sample_bin_to_tint/nwb_test_bin.nwb!
root pynwb.file.NWBFile at 0x139924453646400
Fields:
  acquisition: {
    ElectricalSeries_raw <class 'pynwb.ecephys.ElectricalSeries'>
  }
  devices: {
    Axona <class 'pynwb.device.Device'>
  }
  electrode_groups: {
    Group0 <class 'pynwb.ecephys.ElectrodeGroup'>,
    Group1 <class 'pynwb.ecephys.ElectrodeGroup'>,
    Group2 <class 'pynwb.ecephys.ElectrodeGroup'>,
    Group3 <class 'pynwb.ecephys.ElectrodeGroup'>
  }
  electrodes: electrodes <class 'hdmf.common.table.DynamicTable'>
  experimenter: ['Abid']
  file_create_date: [datetime.datetime(2021, 8, 7, 12, 56, 11, 66643, tzinfo=tzoffset(None, 7200))]
  identifier: 32303dbe-e9c6-4232-81bb-9cfb4bc7fab3
  processing: {
    behavior <class 'pynwb.base.ProcessingModule'>,
    ecephys <class 'pynwb.base.ProcessingModule'>
  }
  session_start_time: 2020-10-04 11:07:07+02:00
  timestamps_reference_time: 2020-10-04 11:07:07+0

### Axona raw (`.bin` + `.set`)

In [ ]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

In [ ]:
import os
from pathlib import Path

# Old .bin file
base_dir = Path('/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/')
dir_name = base_dir / 'example_data_raw'
base_filename = '20201004_Raw'

# New .bin file
base_dir = Path('/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/new_session_data')
dir_name = base_dir / '06172021-HPC-B6-RAW'
base_filename = '06172021-HPC-B6-RAW'
filename = os.path.join(dir_name, base_filename)
set_file = filename + '.set'
bin_file = filename + '.bin'
print(filename)

In [ ]:
from nwb_conversion_tools import (
    NWBConverter, AxonaRecordingExtractorInterface, AxonaPositionDataInterface
)

In [ ]:
class HussainiAxonaNWBConverter(NWBConverter):
    data_interface_classes = dict(
        AxonaRecordingExtractorInterface=AxonaRecordingExtractorInterface,
        AxonaPositionDataInterface=AxonaPositionDataInterface
    )

In [ ]:
import random
import string
from typing import Union, Optional
from pathlib import Path
import spikeextractors as se
from pynwb import NWBFile
import numpy as np
import re
import datetime
import json
from jsonschema import validate, ValidationError

from nwb_conversion_tools.utils.json_schema import get_schema_from_method_signature, get_base_schema, fill_defaults
from nwb_conversion_tools import SpikeGLXRecordingInterface
from nwb_conversion_tools.datainterfaces.ecephys.axona.axonadatainterface import parse_generic_header

In [ ]:
# Specify source data

source_data = dict(
    AxonaPositionDataInterface=dict(
        filename=set_file
    ),
    AxonaRecordingExtractorInterface=dict(
        filename=set_file
    )
)
print(json.dumps(source_data, indent=2))

In [ ]:
# Initialize HussainiAxonaNWBConverter

converter = HussainiAxonaNWBConverter(source_data=source_data)

In [ ]:
# Get metadata

metadata = converter.get_metadata()
metadata

In [ ]:
# Get metadata_schema from converter

metadata_schema = converter.get_metadata_schema()

print(json.dumps(metadata_schema['properties'], indent=2))

In [ ]:
# Validate metadata against metadata_schema

validate(
    instance=converter.get_metadata(),
    schema=converter.get_metadata_schema()
)

In [ ]:
converter.get_metadata()

In [ ]:
from nwb_conversion_tools.utils.conversion_tools import (
    get_default_nwbfile_metadata, make_nwbfile_from_metadata
)

In [ ]:
metadata

In [ ]:
output_file = base_dir / 'out_example.nwb'

In [ ]:
output_file

In [ ]:
# Export to NWB file

output_file = base_dir / 'out_example.nwb'

converter.run_conversion(
    metadata=metadata,
    nwbfile_path=output_file,
    overwrite=True,
    save_to_file=True,
    conversion_options=None
)

In [ ]:
# Check NWB file

from pynwb import NWBHDF5IO

fname = output_file
with NWBHDF5IO(fname, 'r') as io:
    nwbfile = io.read()
    print(nwbfile)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt


from nwbwidgets import nwb2widget

output_file = base_dir / 'out_example.nwb'

io = NWBHDF5IO(output_file, mode='r')
nwb = io.read()

nwb2widget(nwb)

In [ ]:
from nwb_conversion_tools.datainterfaces.ecephys.axona.axonadatainterface import get_position_object

pos = get_position_object(filename=set_file)

In [ ]:
nwb.processing['behavior'].data_interfaces['Position'].spatial_series['t'].data

In [ ]:
pos.spatial_series['t'].timestamps

### Parse `.pos` files

In [ ]:
from nwb_conversion_tools.datainterfaces.ecephys.axona.axonadatainterface import parse_generic_header

# Old pos file
pos_file = '/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/sample_bin_to_tint/axona_sample.pos'

# New pos file
pos_file = '/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/new_session_data/06172021-HPC-B6-UNIT/06172021-HPC-B6-UNIT/06172021-B6-HPC-UNIT.pos'
print(pos_file)

pos_header = parse_generic_header(pos_file, None)
pos_header

Data (with memory map)

In [ ]:
def get_header_bstring(file):
    """
    Scan file for the occurrence of 'data_start' and return the header
    as byte string

    Parameters
    ----------
    file (str or path): file to be loaded

    Returns
    -------
    str: header byte content
    """

    header = b''
    with open(file, 'rb') as f:
        for bin_line in f:
            if b'data_start' in bin_line:
                header += b'data_start'
                break
            else:
                header += bin_line
    return header

In [ ]:
with open(pos_file, 'rb') as f:
    print(f.read(532))
    print(f.read(1))

### LFP data

First, let's look at the tools from the Hussaini lab. And let's also remind ourselves what the file formats of .eeg and .egf data should look like:

```
EEG data is usually recorded continuously at 250 Hz in unit recording mode. The “.eeg” and “.eg2” files contain the data from the primary and secondary EEG channels, if these have been enabled. Very simply, the data consist of “num_EEG_samples” data bytes, following on from the data_start. The sample count is specified in the header. The “.egf” file is stored if a user selects a higher-sample rate EEG. Samples are normally collected at 4800 Hz (specified in the header), and are also normally 2 bytes long, rather than just 1.
```

In [ ]:
# Hussaini lab tools: https://github.com/HussainiLab/BinConverter/blob/master/BinConverter/core/CreateEEG.py

from BinConverter.core.CreateEEG import (
    fir_hann, fir_hann, EEG_downsample, create_egf, write_eeg
)

Try using a NumpyRecordingExtractor to ingest '.eeg' or '.egf' data.
The user can ultimately specify whether to use .eeg or .egf data to be saved as LFP data in the nwb file. By default we
will prefer .eeg data, since it has a lower sampling rate and is more memory efficient. In most cases researchers will
not want to look at high frequency oscillations in LFP data.

We can first extract the data of interest as a numpy ndarray, then create a NumpyRecordingExtractor for the nwb conversion.


*  Case 1: Extract numpy ndarray from `.bin` data (very high Fs)

In [ ]:
not_scaled = 100
gain = 1.25
offset = 10
scaled = not_scaled * gain + offset

print(scaled)

unscaled = (scaled - offset) / gain

print(unscaled)

In [ ]:
recording_filt.get_channel_gains()

In [ ]:
scaled = not_scaled * gain + offset

non_scales = (scaled - offset) / gain

In [ ]:
# scaling
channel_ids = recording.get_channel_ids()
traces = recording.get_traces(start_frame=0, end_frame=100, return_scaled=False)

channel_idxs = np.array([recording.get_channel_ids().index(ch) for ch in channel_ids])
gains = recording.get_channel_gains()[channel_idxs, None]
offsets = recording.get_channel_offsets()[channel_idxs, None]
unscaled = (scaled - offset) / gain

traces = (traces.astype("float32") * gains + offsets).astype("float32")

In [ ]:
recording.get_traces(start_frame=0, end_frame=100, return_scaled=False)

In [ ]:
traces

In [ ]:
se.CacheRecordingExtractor??

In [ ]:
# Filter the data to avoid aliasing based on desired sampling frequency. The nyquest theorem states we need 
# twice the sampling rate of the highest frequency we are interested in. Conversely, we want to filter out all frequencies
# higher than half our sampling rate. Since filters are not perfect and have a certain transition width until full
# attenuation, we are conservative and choose sampling_rate / 2.25 as our cut-off high frequency threshold.

import spiketoolkit as st

sampling_rate = 250  # .eeg=250, .egf=4800

recording_filt = st.preprocessing.bandpass_filter(recording, freq_min=0, freq_max=np.floor(sampling_rate / 2.25))

#recording_downsamp = st.preprocessing.ResampleRecording(recording_filt, sampling_rate)

channel_ids = recording.get_channel_ids()
channel_idxs = np.array([recording.get_channel_ids().index(ch) for ch in channel_ids])
gains = recording.get_channel_gains()[channel_idxs, None]
offsets = recording.get_channel_offsets()[channel_idxs, None]
unscaled = (recording_filt.get_traces() - offsets) / gains

traces = (traces.astype("float32") * gains + offsets).astype("float32")

In [ ]:
# Filter the data to avoid aliasing based on desired sampling frequency. The nyquest theorem states we need 
# twice the sampling rate of the highest frequency we are interested in. Conversely, we want to filter out all frequencies
# higher than half our sampling rate. Since filters are not perfect and have a certain transition width until full
# attenuation, we are conservative and choose sampling_rate / 2.25 as our cut-off high frequency threshold.

import spiketoolkit as st

sampling_rate = 250  # .eeg=250, .egf=4800

recording_filt = st.preprocessing.bandpass_filter(recording, freq_min=0, freq_max=np.floor(sampling_rate / 2.25))

recording_downsamp = st.preprocessing.ResampleRecording(recording_filt, sampling_rate)

In [ ]:
perma_cache_filename = os.path.join(dir_name, 'cached_data_eeg.dat') 
recording_cache = se.CacheRecordingExtractor(recording_filt, save_path = perma_cache_filename)
recording_cache.dump_to_pickle(os.path.join(dir_name, 'cached_data_eeg.pkl'))

In [ ]:
recording_cache = se.load_extractor_from_pickle(os.path.join(dir_name, 'cached_data_eeg.pkl'))

In [ ]:
type(recording_cache)

*  Case 2: Extract numpy ndarray from `.eeg` data (low Fs)

I believe I can simply extract the int8 or int16 data from an .eeg or .egf file, which corresponds to a particular EEG channel, and put it into a numpy recordingextractor (like Ben suggested). Then I only need to include the gain information for each channel and the metadata and I should be done. 

In [ ]:
def get_eeg_sampling_frequency(filename):
    """
    Read sampling frequency from .eegX or .egfX file header.
    
    Parameters:
    -----------
    filename : Path or str
        Full filename of Axona `.eegX` or `.egfX` file.
        
    Returns:
    --------
    Fs : int
        Sampling frequency
    """
    Fs_entry = parse_generic_header(eeg_fname, ['sample_rate'])
    Fs = int(float(Fs_entry.get('sample_rate').split(' ')[0]))

    return Fs

In [ ]:
# Function for reading .eegX or .egfX data (unscaled)

from nwb_conversion_tools.datainterfaces.ecephys.axona.axonadatainterface import get_header_bstring

def read_eeg_file_lfp_data(filename):
    """
    Read LFP data from Axona `.eegX` or `.egfX` file.

    Parameters:
    -------
    filename (Path or Str):
        Full filename of Axona `.eegX` or `.egfX` file.

    Returns:
    -------
    np.memmap (nobs x 1)
    """
    
    lfp_dtype = '>i1'
    footer_size = len('\r\ndata_end\r\n')
    header_size = len(get_header_bstring(filename))
    num_bytes = os.path.getsize(filename) - header_size - footer_size

    # .eeg files are int8, .egf files are int16
    if str(filename).split('.')[1][0:3] == 'egf':
        lfp_dtype = '>i2'
        num_bytes = num_bytes // 2

    eeg_data = np.memmap(
        filename=filename,
        dtype=lfp_dtype,
        mode='r',
        offset=len(get_header_bstring(filename)),
        shape=(1, num_bytes),
    )

    return eeg_data

In [ ]:
def get_all_filenames(filename):
    """
    Read LFP filenames of `.eeg` or `.egf` files in filename's directory.
    E.g. if filename='/my/directory/my_file.eeg', all .eeg channels will be
    appended to the output.
    
    Parameters:
    -----------
    filename : path-like
        Full filename of either .egg or .egf file
        
    Returns:
    --------
    path_list : list
        List of filenames
    """
    
    suffix = Path(filename).suffix[0:4]
    current_path = Path(filename).parent

    path_list = [cur_path.name for cur_path in Path(filename).parent.rglob('*' + suffix + '*')]

    return path_list

In [ ]:
def read_all_eeg_file_lfp_data(filename):
    """
    Read LFP data from all Axona `.eeg` or `.egf` files in filename's directory.
    E.g. if filename='/my/directory/my_file.eeg', all .eeg channels will be conactenated
    to a single np.array (chans x nobs). For .egf files substitude the file suffix.

    Parameters:
    -------
    filename (Path or Str):
        Full filename of Axona `.eeg` or `.egf` file.

    Returns:
    -------
    np.array (chans x obs)
    """

    filename_list = get_all_filenames(filename)
    parent_path = Path(filename).parent

    eeg_memmaps = list()
    sampling_rates = set()
    for fname in filename_list:

        sampling_rates.add(get_eeg_sampling_frequency(parent_path / fname))
        
        eeg_memmaps.append(read_eeg_file_lfp_data(parent_path / fname))

    assert len(sampling_rates) < 2, 'File headers specify different sampling rates. Cannot combine EEG data.'
    
    eeg_data = np.concatenate(eeg_memmaps, axis=0)
    
    return eeg_data

In [ ]:
from nwb_conversion_tools.datainterfaces.ecephys.baselfpextractorinterface import BaseLFPExtractorInterface
from nwb_conversion_tools.datainterfaces.ecephys.axona.axonadatainterface import AxonaRecordingExtractorInterface
from nwb_conversion_tools.utils.json_schema import get_schema_from_hdmf_class
from nwb_conversion_tools.utils.spike_interface import write_recording
from pynwb.ecephys import ElectricalSeries
import spiketoolkit as st


OptionalPathType = Optional[Union[str, Path]]


def AxonaLFPNumpyExtractorWrapper(filename):
    """
    Wrapper for instantiating a NumpyRecordingExtractor given an `.eeg` or `.egf` filename.
    """
    return se.NumpyRecordingExtractor(
        timeseries=read_all_eeg_file_lfp_data(filename),
        sampling_frequency=get_eeg_sampling_frequency(filename)
    )

class AxonaLFPDataInterface(AxonaRecordingExtractorInterface):
    """ ... """

    @classmethod
    def get_source_schema(cls):
        return dict(
            required=['filename'],
            properties=dict(
                filename=dict(
                    type='string'
                )
            ),
            type='object',
            additionalProperties=False
        )
    
    def __init__(self, **source_data):
        self.recording_extractor = AxonaLFPNumpyExtractorWrapper(filename)
        self.subset_channels = None
        self.source_data = source_data
        
    def get_metadata_schema(self):
        metadata_schema = super().get_metadata_schema()
        metadata_schema["properties"]["Ecephys"]["properties"].update(
            ElectricalSeries_lfp=get_schema_from_hdmf_class(ElectricalSeries)
        )
        return metadata_schema

    def get_metadata(self):
        """Retrieve Ecephys metadata specific to the Axona format."""
        metadata = super().get_metadata()
        metadata['Ecephys'].pop('ElectricalSeries_raw', None)
        metadata['Ecephys'].update(
            ElectricalSeries_lfp=dict(
                name="LFP",
                description="Local field potential signal."
            )
        )

        return metadata
    
    def run_conversion(
        self,
        nwbfile: NWBFile,
        metadata: dict = None,
        stub_test: bool = False,
        use_times: bool = False,
        save_path: OptionalPathType = None,
        overwrite: bool = False,
        buffer_mb: int = 500
    ):
        """
        Primary function for converting low-pass recording extractor data to nwb.

        Parameters
        ----------
        nwbfile: NWBFile
            nwb file to which the recording information is to be added
        metadata: dict
            metadata info for constructing the nwb file (optional).
            Should be of the format
                metadata['Ecephys']['ElectricalSeries'] = dict(name=my_name, description=my_description)
        use_times: bool
            If True, the times are saved to the nwb file using recording.frame_to_time(). If False (default),
            the sampling rate is used.
        save_path: PathType
            Required if an nwbfile is not passed. Must be the path to the nwbfile
            being appended, otherwise one is created and written.
        overwrite: bool
            If using save_path, whether or not to overwrite the NWBFile if it already exists.
        stub_test: bool, optional (default False)
            If True, will truncate the data to run the conversion faster and take up less memory.
        buffer_mb: int (optional, defaults to 500MB)
            Maximum amount of memory (in MB) to use per iteration of the internal DataChunkIterator.
            Requires trace data in the RecordingExtractor to be a memmap object.
        """
        if stub_test or self.subset_channels is not None:
            recording = self.subset_recording(stub_test=stub_test)
        else:
            recording = self.recording_extractor
        write_recording(
            recording=recording,
            nwbfile=nwbfile,
            metadata=metadata,
            use_times=use_times,
            write_as="lfp",
            es_key="ElectricalSeries_lfp",
            save_path=save_path,
            overwrite=overwrite,
            buffer_mb=buffer_mb
        )

In [ ]:
# I cannot get it to work with a child of BaseLFPExtractorInterface
'''
from nwb_conversion_tools.datainterfaces.ecephys.baselfpextractorinterface import BaseLFPExtractorInterface
from nwb_conversion_tools.utils.json_schema import get_schema_from_hdmf_class
from pynwb.ecephys import ElectricalSeries


def AxonaLFPNumpyExtractorWrapper(filename):
    """
    Wrapper for instantiating a NumpyRecordingExtractor given an `.eeg` or `.egf` filename.
    """
    return se.NumpyRecordingExtractor(
        timeseries=read_all_eeg_file_lfp_data(filename),
        sampling_frequency=get_eeg_sampling_frequency(filename)
    )

class AxonaLFPDataInterface(BaseLFPExtractorInterface):
    """ ... """

    RX = se.NumpyRecordingExtractor

    @classmethod
    def get_source_schema(cls):
        return dict(
                    required=['filename'],
                    properties=dict(
                        filename=dict(
                            type='string'
                        )
                    ),
                    type='object',
                    additionalProperties=False
                )
    
    def __init__(self, **source_data):
        self.recording_extractor = AxonaLFPNumpyExtractorWrapper(filename)
        self.subset_channels = None
        self.source_data = source_data
        
    def get_metadata_schema(self):
        metadata_schema = super().get_metadata_schema()
        metadata_schema["properties"]["Ecephys"]["properties"].update(
            ElectricalSeries_lfp=get_schema_from_hdmf_class(ElectricalSeries)
        )
        return metadata_schema

    def get_metadata(self):

        # Extract information for specific parameters from .set file
        params_of_interest = ["experimenter", "comments", "duration", "sw_version"]
        set_file = self.source_data["filename"].split(".")[0] + ".set"
        par = parse_generic_header(set_file, params_of_interest)

#        # assign unique group to each channel, and fill in group_name property
#        RX.set_channel_groups(
#            groups=RX.get_channel_ids(),
#            channel_ids=RX.get_channel_ids()
#        )
#        unique_elec_group_names = set(RX.get_channel_groups())

#        for channel_id in RX.get_channel_ids():
#            RX.set_channel_property(
#                channel_id=channel_id,
#                property_name="group_name",
#                value=f"Group{channel_id}"
#            )

        # Add available metadata
        metadata = super().get_metadata()
        metadata["NWBFile"] = dict(
            session_start_time=read_axona_iso_datetime(set_file),
            session_description=par["comments"],
            experimenter=[par["experimenter"]],
        )

        metadata["Ecephys"] = dict(
            Device=[
                dict(
                    name="Axona",
                    description="Axona DacqUSB, sw_version={}".format(par["sw_version"]),
                    manufacturer="Axona",
                ),
            ],
            ElectrodeGroup=[
                dict(
                    name=f"Group{group_name}",
                    location="",
                    device="Axona",
                    description=f"Group {group_name} electrodes.",
                )
                for group_name in unique_elec_group_names
            ],
            Electrodes=[
                dict(
                    name='group_name',
                    description="The name of the ElectrodeGroup this electrode is a part of."
                )
            ],
            ElectricalSeries_lfp=dict(
                name="ElectricalSeries_lfp",
                description="EEG (lfp) acquisition traces."
            ),
        )
        
        return metadata
'''

In [ ]:
from nwb_conversion_tools.datainterfaces.ecephys.axona.axonadatainterface import read_axona_iso_datetime

In [ ]:
eeg_fname = '/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/Axona_Tint_1ms/20201004_Tint.eeg'

In [ ]:
filename = eeg_fname

In [ ]:
lfp_interface = AxonaLFPDataInterface(filename=eeg_fname)

In [ ]:
class HussainiAxonaNWBConverter(NWBConverter):
    data_interface_classes = dict(
        #AxonaRecordingExtractorInterface=AxonaRecordingExtractorInterface,
        AxonaPositionDataInterface=AxonaPositionDataInterface,
        AxonaLFPDataInterface=AxonaLFPDataInterface
    )

In [ ]:
# Specify source data

source_data = dict(
    AxonaPositionDataInterface=dict(
        filename=eeg_fname.replace('.eeg', '.set')
    ),
    #AxonaRecordingExtractorInterface=dict(
    #    filename=eeg_fname.replace('.eeg', '.set')
    #),
    AxonaLFPDataInterface=dict(
        filename=eeg_fname
    )
)
print(json.dumps(source_data, indent=2))


# Instantiate converter
converter = HussainiAxonaNWBConverter(source_data=source_data)

In [ ]:
validate(
    instance=lfp_interface.get_metadata(),
    schema=lfp_interface.get_metadata_schema()
)

In [ ]:
converter = HussainiAxonaNWBConverter(source_data=source_data)


# Get metadata
metadata = converter.get_metadata()
metadata


# Get metadata_schema from converter
metadata_schema = converter.get_metadata_schema()

print(json.dumps(metadata_schema['properties'], indent=2))


# Validate metadata against metadata_schema
validate(
    instance=converter.get_metadata(),
    schema=converter.get_metadata_schema()
)


converter.get_metadata()


# Export to NWB file
output_file = base_dir / 'out_example.nwb'

converter.run_conversion(
    metadata=metadata, 
    nwbfile_path=output_file,
    overwrite=True,
    save_to_file=True,
    conversion_options=None
)

In [ ]:
# Check NWB file

from pynwb import NWBHDF5IO

output_file = base_dir / 'out_example.nwb'

fname = output_file
with NWBHDF5IO(fname, 'r') as io:
    nwbfile = io.read()
    print(nwbfile)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

from nwbwidgets import nwb2widget

output_file = base_dir / 'out_example.nwb'

io = NWBHDF5IO(output_file, mode='r')
nwb = io.read()

nwb2widget(nwb)

The Axona format comes in the form of the raw data (`.bin` + `.set`) or Unit data (`.X`, `.set`, `.eegX` or `.egfX`, `.pos`). As such, a datainterface has to be somewhat flexible. 

We decided that it is not worth the effort to allow for a direct conversion from raw data to `nwb` including EEG and position data, since the Hussaini lab can already convert raw data to EEG and pos respectively. In addition, the converter for position data could also be used for `Intan` data, if the lab converts `Intan` to `.pos` themselves. 

Regarding the Unit data, I am not sure how or whether to add a datainterface that converts to `nwb` directly. As it stands the converted data would include the Gaussian noise added by the `AxonaUnitRecordingExtractor`. I do not really think that this is useful. In addition, if desired one could simply use the syntax from the tutorial notebook to export unit data to nwb.

In general, lab specific data converters are a combination of multiple data interfaces. But in this case it might really depend on which data types the user has available and wants to convert. 

Currently, we can convert `.bin` file ecephys data, `.eeg` or `.egf` file data and `.pos` file data.

In [ ]:
# Add a datainterface for unit data just for good measure

class AxonaUnitRecordingExtractorInterface(AxonaRecordingExtractorInterface):
    """Primary data interface class for converting a AxonaRecordingExtractor"""

    RX = se.AxonaUnitRecordingExtractor

    @classmethod
    def get_source_schema(cls):
        return get_schema_from_method_signature(cls.__init__)

    def __init__(self, **source_data):
        self.recording_extractor=se.AxonaUnitRecordingExtractor(**source_data)
        self.subset_channels = None
        self.source_data = source_data

In [ ]:
# Add a datainterface for unit data just for good measure

class AxonaUnitRecordingExtractorInterface(AxonaRecordingExtractorInterface):
    """Primary data interface class for converting a AxonaRecordingExtractor"""

    RX = se.AxonaUnitRecordingExtractor

    @classmethod
    def get_source_schema(cls):
        return dict(
            required=['filename'],
            properties=dict(
                filename=dict(
                    type='string'
                ),
            ),
            type='object',
            additionalProperties=True
        )

    def __init__(self, filename: str, noise_std=3.5):
        super().__init__(filename=filename)
        self.recording_extractor=se.AxonaUnitRecordingExtractor(filename=filename, noise_std=noise_std)

In [ ]:
class HussainiAxonaNWBConverter(NWBConverter):
    data_interface_classes = dict(
        AxonaUnitRecordingExtractorInterface=AxonaUnitRecordingExtractorInterface,
        AxonaPositionDataInterface=AxonaPositionDataInterface,
        AxonaLFPDataInterface=AxonaLFPDataInterface
    )
    
# Specify source data
source_data = dict(
    AxonaPositionDataInterface=dict(
        filename=eeg_fname.replace('.eeg', '.set')
    ),
    AxonaUnitRecordingExtractorInterface=dict(
        filename=eeg_fname.replace('.eeg', '.set'),
        noise_std=3.5
    ),
    AxonaLFPDataInterface=dict(
        filename=eeg_fname
    )
)
print(json.dumps(source_data, indent=2))


# Instantiate converter
converter = HussainiAxonaNWBConverter(source_data=source_data)


# Get metadata
metadata = converter.get_metadata()
metadata


# Get metadata_schema from converter
metadata_schema = converter.get_metadata_schema()
print(json.dumps(metadata_schema['properties'], indent=2))


# Validate metadata against metadata_schema
validate(
    instance=converter.get_metadata(),
    schema=converter.get_metadata_schema()
)


# Export to NWB file
output_file = base_dir / 'out_example3.nwb'

converter.run_conversion(
    metadata=metadata, 
    nwbfile_path=output_file,
    overwrite=True,
    save_to_file=True,
    conversion_options=None
)

In [ ]:
# Check NWB file
from pynwb import NWBHDF5IO

fname = output_file
with NWBHDF5IO(fname, 'r') as io:
    nwbfile = io.read()
    print(nwbfile)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

from nwbwidgets import nwb2widget

output_file = base_dir / 'out_example3.nwb'

io = NWBHDF5IO(output_file, mode='r')
nwb = io.read()

nwb2widget(nwb)

In [ ]:
output_file_test = '/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/new_session_data/out_example2.nwb'

In [ ]:
axona_unit_interface.run_conversion??

In [ ]:
metadata = axona_unit_interface.get_metadata()

nwb_test = axona_unit_interface.run_conversion(
    nwbfile=None,
    save_path=output_file_test,
    metadata=metadata
)

In [ ]:
# Read .eeg data with Hussaini lab tool

import mmap
import contextlib

def ReadEEG(eeg_fname):
    """input:
    eeg_filename: the fullpath to the eeg file that is desired to be read.
    Example: C:\Location\of\eegfile.eegX
    Output:
    The EEG waveform, and the sampling frequency"""

    with open(eeg_fname, 'rb') as f:

        is_eeg = False
        if 'eeg' in eeg_fname:
            is_eeg = True
            # Fs = 250
        # else:
        #    Fs = 4.8e3

        with contextlib.closing(mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ)) as m:
            # find the data_start
            start_index = int(m.find(b'data_start') + len('data_start'))  # start of the data
            stop_index = int(m.find(b'\r\ndata_end'))  # end of the data

            sample_rate_start = m.find(b'sample_rate')
            sample_rate_end = m[sample_rate_start:].find(b'\r\n')
            Fs = float(m[sample_rate_start:sample_rate_start + sample_rate_end].decode('utf-8').split(' ')[1])

            m = m[start_index:stop_index]

            if is_eeg:
                EEG = np.fromstring(m, dtype='>b')
            else:
                EEG = np.fromstring(m, dtype='<h')

            return EEG, int(Fs)

In [ ]:
eeg_fname = '/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/Axona_Tint_1ms/20201004_Tint.eeg'

eeg = read_eeg_file_lfp_data(eeg_fname)

In [ ]:
eeg_huss = ReadEEG(eeg_fname)

In [ ]:
print(eeg.shape)
print(eeg[0:100])

In [ ]:
print(eeg_huss[0].shape)
eeg_huss[0][0:100]

In [ ]:
def get_header_bstring(file):
    """
    Scan file for the occurrence of 'data_start' and return the header
    as byte string

    Parameters
    ----------
    file (str or path): file to be loaded

    Returns
    -------
    str: header byte content
    """
    header = b''
    with open(file, 'rb') as f:
        for bin_line in f:
            if b'data_start' in bin_line:
                header += b'data_start'
                break
            else:
                header += bin_line
    return header

In [ ]:
set_filename

In [ ]:
#plt.plot(recording.neo_reader.get_analogsignal_chunk(channel_indexes=[15])[0:4800])
plt.plot(recording.neo_reader.get_analogsignal_chunk(channel_indexes=[16])[0:4800])
plt.plot(recording.neo_reader.get_analogsignal_chunk(channel_indexes=[17])[0:4800])
plt.plot(recording.neo_reader.get_analogsignal_chunk(channel_indexes=[18])[0:4800])
plt.plot(recording.neo_reader.get_analogsignal_chunk(channel_indexes=[19])[0:4800])

In [ ]:
import scipy as sci

f, Pxx_den = sci.signal.welch(
    np.squeeze(recording.neo_reader.get_analogsignal_chunk(channel_indexes=[19])),
    48000,
    nperseg=1024
)
plt.semilogy(f, Pxx_den)
plt.xlim([0, 5000])

# If this is how the eeg data is saved in the .bin file, what is the point of the egf sampling rate of 4800 Hz?!
# It looks like the data is already lowpass filtered at approximately 1000 Hz.

In [ ]:
#https://github.com/HussainiLab/BinConverter/blob/master/BinConverter/core/Tint_Matlab.py
from BinConverter.core.Tint_Matlab import get_active_eeg

set_file_eeg = '/mnt/c/tmp_data/catalystneuro/sample_bin_to_tint/axona_sample.set'
bin_file_eeg = '/mnt/c/tmp_data/catalystneuro/sample_bin_to_tint/axona_sample.bin'

active_eeg_channels = get_active_eeg(set_file_eeg)
active_eeg_channels

In [ ]:
active_eeg_channel_numbers = np.asarray(list(active_eeg_channels.values())) + 1
active_eeg_channel_numbers

In [ ]:
for channel in active_eeg_channel_numbers:

    for eeg_number, eeg_chan_value in sorted(active_eeg_channels.items()):
        if eeg_chan_value == channel - 1:
            
            print(eeg_number, eeg_chan_value)

In [ ]:
from BinConverter.core.readBin import get_bin_data

EEG = get_bin_data(bin_file_eeg, channels=[channel])

EEG.shape

In [ ]:
rx.get_traces?

In [ ]:
channel

In [ ]:
rx = se.AxonaRecordingExtractor(filename=bin_file_eeg)

rx.get_traces(return_scaled=False)

In [ ]:
EEG

In [ ]:
create_eeg(eeg_filename, EEG, Fs, DC_Blocker=self.dc_blocker.isChecked())

In [ ]:
eeg_length = eeg_data.shape[0]
print(eeg_length)
print(eeg_data.shape)

In [ ]:
eeg_huss[0][0:eeg_length].shape

In [ ]:
np.array_equal(eeg_data, eeg_huss[0])

In [ ]:
eeg_huss[0].shape

*  Case 3: Extract numpy ndarray from `.egf` data (high Fs)

In [ ]:

RX = se.NumpyRecordingExtractor()

RX

In [ ]:
se.CacheRecordingExtractor??

In [ ]:
from nwb_conversion_tools.datainterfaces.ecephys.baselfpextractorinterface import BaseLFPExtractorInterface
from nwb_conversion_tools.datainterfaces.ecephys.axona.axonadatainterface import AxonaRecordingExtractorInterface
from nwb_conversion_tools.utils.json_schema import get_schema_from_hdmf_class
from nwb_conversion_tools.utils.spike_interface import write_recording
from pynwb.ecephys import ElectricalSeries
import spiketoolkit as st


OptionalPathType = Optional[Union[str, Path]]


class AxonaLFPExtractorInterface(AxonaRecordingExtractorInterface):
    """..."""
    
    def __init__(self, filename: str):
        super().__init__(filename=filename)
        sampling_rate = 250  # .eeg=250, .egf=4800
        print('Filtering and caching, this may take a few minutes ...')
        recording_filt = st.preprocessing.bandpass_filter(
            self.RX(filename=filename),
            freq_min=0,
            freq_max=np.floor(sampling_rate / 2.25)
        )
        recording_downsamp = st.preprocessing.ResampleRecording(recording_filt, sampling_rate)
        recording_downsamp
        self.recording_extractor = se.CacheRecordingExtractor(
            recording_downsamp, return_scaled=False
        )
        self.subset_channels = None
        
    def get_metadata_schema(self):
        metadata_schema = super().get_metadata_schema()
        metadata_schema["properties"]["Ecephys"]["properties"].update(
            ElectricalSeries_lfp=get_schema_from_hdmf_class(ElectricalSeries)
        )
        return metadata_schema

    def get_metadata(self):
        """Retrieve Ecephys metadata specific to the Axona format."""
        metadata = super().get_metadata()
        metadata['Ecephys'].pop('ElectricalSeries_raw', None)
        metadata['Ecephys'].update(
            ElectricalSeries_lfp=dict(
                name="LFP",
                description="Local field potential signal."
            )
        )

        return metadata
    
    def run_conversion(
        self,
        nwbfile: NWBFile,
        metadata: dict = None,
        stub_test: bool = False,
        use_times: bool = False,
        save_path: OptionalPathType = None,
        overwrite: bool = False,
        buffer_mb: int = 500
    ):
        """
        Primary function for converting low-pass recording extractor data to nwb.

        Parameters
        ----------
        nwbfile: NWBFile
            nwb file to which the recording information is to be added
        metadata: dict
            metadata info for constructing the nwb file (optional).
            Should be of the format
                metadata['Ecephys']['ElectricalSeries'] = dict(name=my_name, description=my_description)
        use_times: bool
            If True, the times are saved to the nwb file using recording.frame_to_time(). If False (default),
            the sampling rate is used.
        save_path: PathType
            Required if an nwbfile is not passed. Must be the path to the nwbfile
            being appended, otherwise one is created and written.
        overwrite: bool
            If using save_path, whether or not to overwrite the NWBFile if it already exists.
        stub_test: bool, optional (default False)
            If True, will truncate the data to run the conversion faster and take up less memory.
        buffer_mb: int (optional, defaults to 500MB)
            Maximum amount of memory (in MB) to use per iteration of the internal DataChunkIterator.
            Requires trace data in the RecordingExtractor to be a memmap object.
        """
        if stub_test or self.subset_channels is not None:
            recording = self.subset_recording(stub_test=stub_test)
        else:
            recording = self.recording_extractor
        write_recording(
            recording=recording,
            nwbfile=nwbfile,
            metadata=metadata,
            use_times=use_times,
            write_as="lfp",
            es_key="ElectricalSeries_lfp",
            save_path=save_path,
            overwrite=overwrite,
            buffer_mb=buffer_mb
        )

In [ ]:
set_filename = '/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/new_session_data/06172021-HPC-B6-RAW/06172021-HPC-B6-RAW.set'

In [ ]:
axo_raw = AxonaRecordingExtractorInterface(filename=set_filename)

print(axo_raw.recording_extractor.get_shared_channel_property_names())

axo_raw.get_metadata()
axo_raw.recording_extractor.get_shared_channel_property_names()

In [ ]:
axo_lfp.source_data

In [ ]:
axo_lfp = AxonaLFPExtractorInterface(filename=set_filename)

print(axo_lfp.recording_extractor.get_shared_channel_property_names())

axo_lfp.get_metadata()
axo_lfp.recording_extractor.get_shared_channel_property_names()

In [ ]:
axo_lfp.get_source_schema()

In [ ]:
axo_lfp.get_conversion_options()

In [ ]:
axo_lfp.get_conversion_options_schema()

In [ ]:
axo_lfp.run_conversion?

In [ ]:
metadata = axo_lfp.get_metadata()

metadata

In [ ]:
import uuid
from datetime import datetime
import warnings
import numpy as np
import distutils.version
from pathlib import Path
from typing import Union, Optional, List
from warnings import warn
import psutil
from collections import defaultdict

import spikeextractors as se
import pynwb
from numbers import Real
from hdmf.data_utils import DataChunkIterator
from hdmf.backends.hdf5.h5_utils import H5DataIO
#from .json_schema import dict_deep_update

PathType = Union[str, Path, None]
ArrayType = Union[list, np.ndarray]

In [ ]:
nwbfile

In [ ]:
class HussainiAxonaNWBConverter(NWBConverter):
    data_interface_classes = dict(
        AxonaRecordingExtractorInterface=AxonaRecordingExtractorInterface,
        AxonaPositionDataInterface=AxonaPositionDataInterface,
        AxonaLFPExtractorInterface=AxonaLFPExtractorInterface
    )
    
# Specify source data
source_data = dict(
#    AxonaPositionDataInterface=dict(
#        filename=set_file
#    ),
#    AxonaRecordingExtractorInterface=dict(
#        filename=set_file
#    ),
    AxonaLFPExtractorInterface=dict(
        filename=set_file
    )
)
print(json.dumps(source_data, indent=2))

# Initialize HussainiAxonaNWBConverter
converter = HussainiAxonaNWBConverter(source_data=source_data)

# Get metadata
metadata = converter.get_metadata()

# Validate metadata against metadata_schema
validate(
    instance=converter.get_metadata(),
    schema=converter.get_metadata_schema()
)

In [ ]:
class HussainiAxonaNWBConverter(NWBConverter):
    data_interface_classes = dict(
        AxonaRecordingExtractorInterface=AxonaRecordingExtractorInterface,
        AxonaPositionDataInterface=AxonaPositionDataInterface,
        AxonaLFPExtractorInterface=AxonaLFPExtractorInterface
    )
    
# Specify source data
source_data = dict(
    AxonaPositionDataInterface=dict(
        filename=set_file
    ),
    AxonaRecordingExtractorInterface=dict(
        filename=set_file
    ),
    AxonaLFPExtractorInterface=dict(
        filename=set_file
    )
)
print(json.dumps(source_data, indent=2))

# Initialize HussainiAxonaNWBConverter
converter = HussainiAxonaNWBConverter(source_data=source_data)

# Get metadata
metadata = converter.get_metadata()

# Validate metadata against metadata_schema
validate(
    instance=converter.get_metadata(),
    schema=converter.get_metadata_schema()
)

# Export to NWB file
output_file = base_dir / 'out_example_test5.nwb'

converter.run_conversion(
    metadata=metadata, 
    nwbfile_path=output_file,
    overwrite=True,
    save_to_file=True,
    conversion_options=None
)

In [ ]:
# Read data from the nwb file to see if we converted everything correctly

io = NWBHDF5IO(output_file, 'r')
nwbfile_in = io.read()

In [ ]:
nwbfile_in.acquisition['ElectricalSeries_raw'].data

In [ ]:
nwbfile_in.processing['ecephys'].data_interfaces['LFP'].electrical_series['LFP'].data

In [ ]:
np.arange(0,100)

In [ ]:
plt.plot(nwbfile_in.acquisition['ElectricalSeries_raw'].data[0:2300, 0])

In [ ]:
plt.plot(nwbfile_in.processing['ecephys'].data_interfaces['LFP'].electrical_series['LFP'].data[0:12, 0])

In [ ]:
test_timeseries_in = nwbfile_in.processing['test_timeseries']
print(test_timeseries_in)

In [ ]:
axo_raw = AxonaRecordingExtractorInterface(filename=set_file)

In [ ]:
nwbfile = generate_nwb

In [ ]:
metadata = axo_raw.get_metadata()

axo_raw.run_conversion(
    nwbfile=make_nwbfile_from_metadata(metadata),
    metadata=metadata,
    overwrite=True,
    write_as='raw'
)

In [ ]:
metadata = axo_lfp.get_metadata()

axo_lfp.run_conversion(
    nwbfile=make_nwbfile_from_metadata(metadata),
    metadata=metadata,
    overwrite=True,
    write_as='lfp'
)

In [ ]:
axo_raw.run_conversion??

In [ ]:
# Export to NWB file
output_file = base_dir / 'out_example_lfp.nwb'

converter.run_conversion(
    metadata=metadata, 
    nwbfile_path=output_file,
    overwrite=True,
    save_to_file=True,
    conversion_options=None
)

In [ ]:
base_dir

In [ ]:
from nwb_conversion_tools.utils.spike_interface import add_electrode_groups

In [ ]:
axo_raw = AxonaRecordingExtractorInterface(filename=filename)

In [ ]:
axo_lfp.recording_extractor.get_channel_property_names(channel_id=0)

In [ ]:
axo_raw.recording_extractor.get_channel_property_names(channel_id=0)

In [ ]:
import uuid
from datetime import datetime
import warnings
import numpy as np
import distutils.version
from pathlib import Path
from typing import Union, Optional, List
from warnings import warn
import psutil
from collections import defaultdict

import spikeextractors as se
import pynwb
from numbers import Real
from hdmf.data_utils import DataChunkIterator
from hdmf.backends.hdf5.h5_utils import H5DataIO
#from .json_schema import dict_deep_update

PathType = Union[str, Path, None]
ArrayType = Union[list, np.ndarray]

In [ ]:
recording = axo_lfp.recording_extractor
#recording = axo_raw.recording_extractor
exclude = ()
nwbfile = make_nwbfile_from_metadata(metadata=metadata)


if nwbfile.electrodes is not None:
    ids_absent = [id not in nwbfile.electrodes.id for id in recording.get_channel_ids()]
    if not all(ids_absent):
        warnings.warn('cannot create electrodes for this recording as ids already exist')

if nwbfile is not None:
    assert isinstance(nwbfile, pynwb.NWBFile), "'nwbfile' should be of type pynwb.NWBFile"
if nwbfile.electrode_groups is None or len(nwbfile.electrode_groups) == 0:
    add_electrode_groups(recording, nwbfile, metadata)
# For older versions of pynwb, we need to manually add these columns
if distutils.version.LooseVersion(pynwb.__version__) < '1.3.0':
    if nwbfile.electrodes is None or 'rel_x' not in nwbfile.electrodes.colnames:
        nwbfile.add_electrode_column('rel_x', 'x position of electrode in electrode group')
    if nwbfile.electrodes is None or 'rel_y' not in nwbfile.electrodes.colnames:
        nwbfile.add_electrode_column('rel_y', 'y position of electrode in electrode group')

defaults = dict(
    x=np.nan,
    y=np.nan,
    z=np.nan,
    # There doesn't seem to be a canonical default for impedence, if missing.
    # The NwbRecordingExtractor follows the -1.0 convention, other scripts sometimes use np.nan
    imp=-1.0,
    location="unknown",
    filtering="none",
    group_name="0"
)
if metadata is None:
    metadata = dict(Ecephys=dict())

if 'Ecephys' not in metadata:
    metadata['Ecephys'] = dict()

if 'Electrodes' not in metadata['Ecephys']:
    metadata['Ecephys']['Electrodes'] = []

assert all([isinstance(x, dict) and set(x.keys()) == set(['name', 'description'])
            for x in metadata['Ecephys']['Electrodes']]), \
    "Expected metadata['Ecephys']['Electrodes'] to be a list of dictionaries, containing the keys 'name' and 'description'"
assert all([x['name'] != 'group' for x in metadata['Ecephys']['Electrodes']]), \
    "Passing metadata field 'group' is deprecated; pass group_name instead!"

if nwbfile.electrodes is None:
    nwb_elec_ids = []
else:
    nwb_elec_ids = nwbfile.electrodes.id.data[:]

elec_columns = defaultdict(dict)  # dict(name: dict(description='',data=data, index=False))
elec_columns_append = defaultdict(dict)
property_names = set()
for chan_id in recording.get_channel_ids():
    for i in recording.get_channel_property_names(channel_id=chan_id):
        property_names.add(i)

# property 'brain_area' of RX channels corresponds to 'location' of NWB electrodes
exclude_names = set(['location','group'] + list(exclude))

channel_property_defaults = {
    list: [],
    np.ndarray: np.array(np.nan),
    str: '',
    Real: np.nan
}
found_property_types = {prop: Real for prop in property_names}

In [ ]:
elec_columns

In [ ]:
recording.get_channel_property_names(channel_id=chan_id)

In [ ]:
axo_lfp.recording_extractor.get_channel_property_names(channel_id=chan_id)

In [ ]:
for prop in property_names:
    prop_skip = False
    if prop not in exclude_names:
        data = []
        prop_chan_count = 0
        # build data:
        for chan_id in recording.get_channel_ids():
            if prop in recording.get_channel_property_names(channel_id=chan_id):
                prop_chan_count += 1
                chan_data = recording.get_channel_property(channel_id=chan_id, property_name=prop)
                # find the type and store (only when the first channel with given property is found):
                if prop_chan_count==1:
                    proptype = [proptype for proptype in channel_property_defaults if isinstance(chan_data, proptype)]
                    if len(proptype) > 0:
                        found_property_types[prop] = proptype[0]
                        # cast as float if any number:
                        if found_property_types[prop]==Real:
                            chan_data = np.float(chan_data)
                        # update data if wrong datatype items filled prior:
                        if len(data) > 0 and not isinstance(data[-1], found_property_types[prop]):
                            data = [channel_property_defaults[found_property_types[prop]]] * len(data)
                    else:
                        prop_skip = True  # skip storing that property if not of default type
                        break
                data.append(chan_data)
            else:
                data.append(channel_property_defaults[found_property_types[prop]])
        # store data after build:
        if not prop_skip:
            index = found_property_types[prop] == ArrayType
            prop_name_new = 'location' if prop == 'brain_area' else prop
            found_property_types[prop_name_new] = found_property_types.pop(prop)
            elec_columns[prop_name_new].update(description=prop_name_new, data=data, index=index)


In [ ]:
elec_columns

In [ ]:
for x in metadata['Ecephys']['Electrodes']:
    elec_columns[x['name']]['description'] = x['description']
    if x['name'] not in list(elec_columns):
        raise ValueError(f'"{x["name"]}" not a property of se object')

# updating default arguments if electrodes table already present:
default_updated = dict()
if nwbfile.electrodes is not None:
    for colname in nwbfile.electrodes.colnames:
        if colname!='group':
            samp_data = nwbfile.electrodes[colname].data[0]
            default_datatype = [proptype for proptype in channel_property_defaults if isinstance(samp_data, proptype)][0]
            default_updated.update({colname:channel_property_defaults[default_datatype]})
default_updated.update(defaults)

for name, des_dict in elec_columns.items():
    des_args = dict(des_dict)
    if name not in default_updated:
        if nwbfile.electrodes is None:
            nwbfile.add_electrode_column(name=name, description=des_args['description'], index=des_args['index'])
        else:
            # build default junk values for data to force add columns later:
            combine_data = [channel_property_defaults[found_property_types[name]]] * len(nwbfile.electrodes.id)
            des_args['data'] = combine_data + des_args['data']
            elec_columns_append[name] = des_args

for name in elec_columns_append:
    _ = elec_columns.pop(name)

for j, channel_id in enumerate(recording.get_channel_ids()):
    if channel_id not in nwb_elec_ids:
        electrode_kwargs = dict(default_updated)
        electrode_kwargs.update(id=channel_id)

        # recording.get_channel_locations defaults to np.nan if there are none
        location = recording.get_channel_locations(channel_ids=channel_id)[0]
        if all([not np.isnan(loc) for loc in location]):
            # property 'location' of RX channels corresponds to rel_x and rel_ y of NWB electrodes
            electrode_kwargs.update(
                dict(
                    rel_x=float(location[0]),
                    rel_y=float(location[1])
                )
            )

        for name, desc in elec_columns.items():
            if name == 'group_name':
                group_name = str(desc['data'][j])
                if group_name!='' and group_name not in nwbfile.electrode_groups:
                    warnings.warn(f"Electrode group {group_name} for electrode {channel_id} was not "
                                  "found in the nwbfile! Automatically adding.")
                    missing_group_metadata = dict(
                        Ecephys=dict(
                            ElectrodeGroup=[dict(
                                name=group_name,
                            )]
                        )
                    )
                    add_electrode_groups(recording, nwbfile, missing_group_metadata)
                electrode_kwargs.update(
                    dict(
                        group=nwbfile.electrode_groups[group_name],
                        group_name=group_name
                    )
                )
            elif 'data' in desc:
                electrode_kwargs[name] = desc['data'][j]

        if 'group_name' not in elec_columns:
            group_id = recording.get_channel_groups(channel_ids=channel_id)[0]
            electrode_kwargs.update(
                dict(
                    group=nwbfile.electrode_groups[str(group_id)],
                    group_name=str(group_id)
                )
            )

        nwbfile.add_electrode(**electrode_kwargs)
# add columns for existing electrodes:
for col_name, cols_args in elec_columns_append.items():
    nwbfile.add_electrode_column(col_name,**cols_args)
assert nwbfile.electrodes is not None, \
    "Unable to form electrode table! Check device, electrode group, and electrode metadata."

In [ ]:
elec_columns

In [ ]:
nwbfile.electrodes[colname].data

In [ ]:
metadata = axo_lfp.get_metadata()

nwbfile = make_nwbfile_from_metadata(metadata=metadata)

axo_lfp.run_conversion(nwbfile, metadata)

# Intan to NWB

For this there are already resources from the Hussaini lab we should be able to use.

See here: ...

In [ ]:
from nwb_conversion_tools import (
    NWBConverter, AxonaRecordingExtractorInterface, AxonaPositionDataInterface, IntanRecordingInterface
)

In [ ]:
from nwb_conversion_tools import IntanRecordingInterface

In [ ]:
class HussainiIntanNWBConverter(NWBConverter):
    data_interface_classes = dict(
        IntanRecordingInterface=IntanRecordingInterface,
    )

In [ ]:
intan_file = '/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/Intan_data/intan_rhd_test_1.rhd'

In [ ]:
# Specify source data

source_data = dict(
    IntanRecordingInterface=dict(
        file_path=intan_file
    )
)
print(json.dumps(source_data, indent=2))

In [ ]:
# Initialize HussainiIntanNWBConverter

intan_converter = HussainiIntanNWBConverter(source_data=source_data)

In [ ]:
# Get metadata_schema from converter

metadata_schema = intan_converter.get_metadata_schema()

print(json.dumps(metadata_schema['properties'], indent=2))

In [ ]:
# Validate metadata against metadata_schema

validate(
    instance=intan_converter.get_metadata(),
    schema=intan_converter.get_metadata_schema()
)